In [2]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import TokenTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
import os
from dotenv import load_dotenv
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import OpenAI
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.llms import Cohere
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
import json
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoModel
from langchain.retrievers.document_compressors import FlashrankRerank

In [3]:

load_dotenv()
COHERE_API_KEY = os.getenv('COHERE_API_KEY')

In [4]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )


In [5]:
loader = DirectoryLoader('../data', glob="*.txt", loader_cls=TextLoader, show_progress=True)
docs = loader.load()
# pretty_print_docs(docs)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 936.86it/s]


In [6]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=256,
    chunk_overlap=64,
)
chunked = text_splitter.split_documents(docs)
# print(len(chunked))
# pretty_print_docs(chunked)

In [7]:

model = AutoModel.from_pretrained('mixedbread-ai/mxbai-embed-large-v1', trust_remote_code=True) 

model_name = "mixedbread-ai/mxbai-embed-large-v1"
model_kwargs = {'device': 'cpu'}
embeddings_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
)

In [8]:
embeddings_model

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='mixedbread-ai/mxbai-embed-large-v1', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={}, multi_process=False, show_progress=False)

In [9]:
# from sentence_transformers import SentenceTransformer

# model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")

In [10]:
%%time
db = FAISS.load_local("mxbai_faiss_index", embeddings_model, allow_dangerous_deserialization=True)

CPU times: user 18.3 ms, sys: 15.4 ms, total: 33.7 ms
Wall time: 65.5 ms


In [11]:
llm = Ollama(model="mistral", temperature=0)
query = "How do I defeat Dreki?"

In [12]:
retriever = db.as_retriever()
docs = retriever.get_relevant_documents(query)
pretty_print_docs(docs)

Document 1:

As you start to deplete the Dreki’s health (approximately two bars), it’ll introduce a slightly new attack into its arsenal that sees the creature leaping from afar, straight into a chomp attack. This attack is indicated by a yellow circle that will appear before it launches the attack.
Don’t be afraid to use your Resurrection Stone if you have one. There’s no checkpoint throughout this fight, so if you happen to die again before the fight is over, you’ll respawn with your stone.
Once the Dreki reaches half health, it’ll introduce another new attack to the battle, where it’ll launch an unlockable (red) attack that sees it rearing up into the air, as it begins to charge an electrical AoE attack. If you’re within the electrical circle after it finishes charging, you’ll be dealt damage and temporarily stunned, unable to move or block incoming attacks.
Higher Difficulty Tip : While the Dreki is charging up its electric attack, use that brief moment as a chance to charge either

In [13]:
%%time
print(llm.invoke(query))

 Dreki is a dragon boss found in the Elder Scrolls V: Skyrim. Defeating Dreki requires preparation, strategy, and knowledge of its strengths and weaknesses. Here are some steps to help you defeat Dreki:

1. Preparation:
   - Ensure your character is well-equipped with appropriate armor, weapons, and potions.
   - Make sure you have a good selection of spells if you're using magic.
   - Save the game before attempting to fight Dreki.

2. Location:
   - Find Dreki's lair in the northern part of Skyrim, near the Throat of the World.
   - Approach the lair cautiously, as there may be dragons guarding the entrance.

3. Strategy:
   - Use ranged attacks or spells to deal damage from a safe distance.
   - If you're using melee, try to avoid getting too close to Dreki's breath attack.
   - Focus on damaging Dreki's weak spots: its wings and head.
   - Use potions and spells to heal yourself during the fight.

4. Tactics:
   - If you have a follower, consider bringing them along for additional 

In [14]:
compressor = FlashrankRerank()

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(query)
pretty_print_docs(compressed_docs)

ms-marco-MultiBERT-L-12.zip: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 98.7M/98.7M [00:11<00:00, 9.03MiB/s]


Document 1:

Higher Difficulty Tip : While the Dreki is charging up its electric attack, use that brief moment as a chance to charge either the Leviathan Axe or the Blades of Chaos to do extra, more impactful damage.
If you happen to escape the attack, just be sure you stay clear of the electrified spots on the ground; otherwise, you’ll also succumb to being electrified – you’ll need to mash circle to escape the hold of the electricity.
----------------------------------------------------------------------------------------------------
Document 2:

Fortunately, the Dreki does not have any unblockable attacks until it reaches half health, so technically speaking, every attack the Dreki throws your way can be blocked; however, this isn’t necessarily very efficient as it makes it quite a bit more challenging to deal damage.
Need some health? Try looking beside the crates that are found along the outskirts of the area.
-----------------------------------------------------------------------

In [15]:
chain = RetrievalQA.from_chain_type(
    llm=llm, retriever=compression_retriever
)

In [16]:
%%time
c2 = chain({"query": query})

/Users/jaxxsulav/Projects/LLM/Andromeda1/LangChain/env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


CPU times: user 2.34 s, sys: 637 ms, total: 2.98 s
Wall time: 7.93 s


In [17]:
print(c2["result"])

 To defeat Dreki, you can block or dodge its saliva attack, and when the creature is stunned, rush in and deal heavy damage using your Leviathan Axe, Blades of Chaos, or Runic Attack. Be careful during the Dreki's electric charge-up as it can electrify the ground, potentially harming you. Additionally, avoid getting hit by its other attacks as much as possible. The Dreki does not have any unblockable attacks until it reaches half health.


In [18]:
%%time
chain = RetrievalQA.from_chain_type(
    llm=Cohere(temperature=0), retriever=compression_retriever, chain_type="refine"
)
c3 = chain({"query": query})

/Users/jaxxsulav/Projects/LLM/Andromeda1/LangChain/env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.cohere.Cohere` was deprecated in langchain-community 0.1.14 and will be removed in 0.2.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import Cohere`.
  warn_deprecated(


CPU times: user 3.05 s, sys: 1.65 s, total: 4.7 s
Wall time: 16.4 s


In [19]:
print(c3["result"])

 To defeat the Dreki, you need to avoid its electric attack by mashing the circle button to escape, then attacking it with your axe or blades. Be careful to avoid the electrified spots on the ground. You can also charge your axe or blades during the brief moment while Dreki is charging its electric attack to do extra damage. You may need to repeat this process multiple times to defeat Dreki. If you are using a shield, you can also parry the Dreki's saliva ball attack by blocking it just before it hits you, which will deal damage and temporarily stun the creature. However, it is not recommended to rely solely on blocking its attacks as this will make dealing damage inefficient. Additionally, you can replenish health from crates located outskirts of the area.


In [20]:
%%time
chain = RetrievalQA.from_chain_type(
    llm=llm, retriever=compression_retriever, chain_type="refine", 
)
c4 = chain({"query": query})

CPU times: user 2.39 s, sys: 599 ms, total: 2.99 s
Wall time: 24.7 s


In [21]:
print(c4["result"])

 To defeat Dreki, you should be prepared for its various attacks, including its saliva ball which can be dodged or blocked with a shield, or even parried back for damage if using the Dauntless Shield. When Dreki charges up its electric attack, use that brief moment to charge your weapon, either the Leviathan Axe or the Blades of Chaos, to deal extra damage. After escaping the attack, avoid the electrified spots on the ground and mash the circle button to escape the hold of electricity.

At the start of the battle, be aware that Dreki will spit a ball of saliva toward you which can be dodged or blocked with a shield. If you choose to block it, you may even parry it back for damage and a temporary stun. Once the creature is stunned, rush in and unleash as much heavy damage as possible before the stun effect runs out. Using your Runic Attack early in the fight is recommended to get the cooldown timer going as soon as possible.

Keep in mind that Dreki does not have any unblockable attacks

In [22]:
%%time
chain = RetrievalQA.from_chain_type(
    llm=llm, retriever=compression_retriever, chain_type="refine", return_source_documents=True
)
c5 = chain({"query": query})

CPU times: user 3.08 s, sys: 1.75 s, total: 4.83 s
Wall time: 23.9 s


In [23]:
print(c5['source_documents'])

[Document(page_content='Higher Difficulty Tip : While the Dreki is charging up its electric attack, use that brief moment as a chance to charge either the Leviathan Axe or the Blades of Chaos to do extra, more impactful damage.\nIf you happen to escape the attack, just be sure you stay clear of the electrified spots on the ground; otherwise, you’ll also succumb to being electrified – you’ll need to mash circle to escape the hold of the electricity.', metadata={'source': '../data/questfortyr.txt', 'relevance_score': 0.96820515}), Document(page_content='Fortunately, the Dreki does not have any unblockable attacks until it reaches half health, so technically speaking, every attack the Dreki throws your way can be blocked; however, this isn’t necessarily very efficient as it makes it quite a bit more challenging to deal damage.\nNeed some health? Try looking beside the crates that are found along the outskirts of the area.', metadata={'source': '../data/questfortyr.txt', 'relevance_score':

In [24]:
print(c5["result"])

 To defeat Dreki, you should be prepared for its various attacks, including its saliva ball which can be dodged or blocked with a shield, or even parried back for damage if using the Dauntless Shield. When Dreki charges up its electric attack, use that brief moment to charge your weapon, either the Leviathan Axe or the Blades of Chaos, to deal extra damage. After escaping the attack, avoid the electrified spots on the ground and mash the circle button to escape the hold of electricity.

At the start of the battle, be aware that Dreki will spit a ball of saliva toward you which can be dodged or blocked with a shield. If you choose to block it, you may even parry it back for damage and a temporary stun. Once the creature is stunned, rush in and unleash as much heavy damage as possible before the stun effect runs out. Using your Runic Attack early in the fight is recommended to get the cooldown timer going as soon as possible.

Keep in mind that Dreki does not have any unblockable attacks

In [26]:
%%time
chain = RetrievalQA.from_chain_type(
    llm=llm, retriever=compression_retriever, chain_type="refine", return_source_documents=True
)
c6 = chain({"query": "Who are you?"})
print(c6['source_documents'])

[Document(page_content='Once you’re happy with your purchases, exit the store and check the nearby store for any items you may have left behind after rigorous battles – these items can include the likes of Forged Iron and Hacksilver.\nBefore leaving to speak with Sindri, you may talk to Brok one final time to receive an optional conversation. Then, once you’re ready, talk with Sindri in his office.\nFollow Sindri to the Workshop\nAfter a very short cutscene, follow Sindri into his workshop, where he’ll reveal his Bifrost machine.\nMeet Brok and Sindri at the Mystic Gateway\nBefore meeting with Brok and Sindri outside, purchase any last items or upgrades – Brok will also offer another optional conversation. Once outside, interact with the gate and watch as Brok and Sindri open the mystic gateway to Svartalfheim with Mimir’s help.\n\nOnce the Mystic Gateway is open, collect Mimir from the Bifrost, and head through the door to travel to Svartalfheim as you seek out the help of Durlin, who

In [27]:
print(c6['result'])

 In God of Kara, I am an assistant designed to help answer questions based on given context or information. In this specific context, you are Kara, and your current objective is to find Durlin in Svartalfheim. To do so, follow Atreus up the ramp from the docking area, and instead of heading towards the large statue of Odin, look to your left where you'll spot a ledge by a water wheel.

To complete the puzzle and reach the next area, jump down the ledge to your right, where you’ll find a Hacksilver pouch and a Red Coffin – this will contain Forged Iron and some Hacksilver. From here, climb the chain dangling from the ledge above, and make your way to the top of the platform. Use your Blades of Chaos to stop the water flowing from the trough.

With the elevator powered up, return to the ledge at the start of the puzzle. Freeze the trough on your left, just before the end – this will allow any overflow to drop onto the wheel below, rising the metal clamp. Once the clamp is raised, you sho